# Assignment 2

## Daniel Henke (176182) & Heinrich Hegenbarth (176168)

The objectives of this assignment are to provide students with a case study that requires both coordination and critical thinking. It also provides an applied set of tasks that will enable you to demonstrate knowledge gathered during the lectures and exercises. A well-known application domain of on-demand electric scooters was chosen to allow the student to focus on the solution in terms of object-oriented programming features of Python.

## Problem Description

The on-demand electric scooter is a service in which electric scooters are made available to individuals at a cost. It allows people to
- pick up an electric scooter from one point and return it at another point.
- The user enters payment information, and the computer unlocks a scooter.
- The user returns the scooter, and it gets locked.
A locked electric scooter can only be used using a proper control mechanism controlled by a mobile application. For many systems, smartphone mapping apps show nearby available scooters. Imagine you have been hired as a software developer for such a project by an on-demand electric scooter service company.

## Task

The functionality of the on-demand electric scooter-based platform is more or less similar to the functionality offered by any other bicycle-sharing platform. Therefore, you can take inspiration from other bicycle-sharing software and make suitable assumptions about the functionality requirements for the electric scooter application. Make suitable assumptions in case of any missing information in the case study. Now, build your application which considers the below events:

1. **Develop an on-demand electric scooter-based platform** using object-oriented features of Python such as assertion, recursion, and polymorphism. Use one sorting algorithm for slot management. Also, implement exception handling and inheritance.
2. **Automate scooter renting and return processes**. Also, focus on the "anywhere" release feature.
3. Individuals registered with the program identify themselves with their membership card ID at any place to check out a scooter for a short period, usually a few hours.
4. Create multiple scenarios of subscribers renting electric scooters while others release electric scooters to stations. Consider a total of 10,000 subscribers, 100 stations, and 5,000 bicycles. Remember, not all electric scooters are in working status all the time.

---

## Assumptions:


### **Asset Management Assumptions**

1. **Asset Types and Pricing**:
   - Only two asset types are available: electric scooters and bicycles.
   - Default pricing and activation fees are hard-coded:
     - Scooters: $0.99 per ride with a $2 activation fee.
     - Bicycles: $0.80 per ride with a $4 activation fee.

2. **Asset Condition and Maintenance**:
   - Assets have a `battery` level, ranging from 0 to 100.
   - Fully charged batteries last 2 hours (120 minutes) of use.
   - Assets have a 1% random failure chance per trip.
   - Repairs are performed at a central location.

4. **Battery and Maintenance Operations**:
   - `chargeAll()` sets battery levels to 100.
   - `repairAll()` repairs all assets needing maintenance, fully recharges them, and relocates them to the central repair location.

---

### **User Assumptions**

1. **User Identification and Payment**:
   - Users require a valid `PaymentMethod` to rent assets.
   - Membership waives the activation fee but requires a valid payment method.

2. **User Interaction with Assets**:
   - Rental begins when a user unlocks an asset and ends when it is locked.
   - Out-of-area returns incur a fee; returns directly to a station offer incentives.

3. **Location-Based Searches**:
   - Users can find the closest available asset or station within a 10 km limit.
   - Search results are limited to the 25 closest assets or stations.

---

### **Asset Behavior and Restrictions**

1. **Unlocking and Locking Assets**:
   - Unlocking requires a battery above 10%, no outstanding repairs, and a locked state.
   - Locking an asset calculates and updates battery levels based on ride time.

2. **Battery Drain and Duration Calculation**:
   - Battery drains at 1% per 1.2 minutes of ride time.
   - Assets become non-operational if the battery reaches 0%.

3. **Out-of-Range Returns**:
   - Returns out of the designated area trigger repairs and add a $100 fee.

---

### **Data Structures and Methods**

1. **Node-Based Linked List for Closest Searches**:
   - The `Node` linked list stores the 25 closest assets or stations, sorted by proximity.
   - `insert` and `shiftNumbering` manage this ordered list.

2. **Station Management**:
   - Each `Station` has a list of parked assets.
   - Closest stations are calculated based on asset return proximity.


## Server Code:

In [10]:
#@title General

#Central object for all the data
assets = {}
stations = {}
users = {}

#statistics variables for later
trips=0
repairs=0
charges=0


#helpful global functions
def changePrice(newPrice, newActication, assetType):
    for asset in assets:
        if assetType=="Scooter":
            asset.scooter_activation=newActication
            asset.scooter_price=newPrice
        elif assetType=="Bicycle":
            asset.bicycle_activation=newActication
            asset.bicycle_price=newPrice


def chargeAll():
    global charges
    #charging only happens at stations or when in need of repair (see below)
    for station in stations.values():
        for asset in station.parked:
            asset.battery=100
            charges+=1

def repairAll():
    global repairs
    for asset in assets.values():
        if asset.needs_repair or (asset.battery<10 and asset.locked):
            asset.needs_repair=False
            asset.battery=100
            #resets them to the main station at Radhuspladsen
            asset.position=Position(12.568337, 55.676098)
            asset.Station_Id=1
            repairs+=1

For easy usage of location data, we created a position class for coordinates.

In [11]:
#@title Position

import math

class Position:
    def __init__(self, x, y):
       # x is the longitude
       # y is the latitude
        self.x=x
        self.y=y

    def __str__(self):
        return f'X: {self.x}, Y: {self.y}'

    def distance(self, other):
       # x is the longitude
       # y is the latitude
        # Convert latitude and longitude from degrees to radians
        x1, y1 = math.radians(self.x), math.radians(self.y)
        x2, y2 = math.radians(other.x), math.radians(other.y)

        # Equirectangular approximation, apparently more stable for short distances
        dx = x2 - x1
        dy = y2 - y1
        avg_lat = (y1 + y2) / 2
        dx_adj = dx * math.cos(avg_lat)

        # Radius of Earth in meters
        r = 6371.0 * 1000
        distance = r * math.sqrt(dx_adj**2 + dy**2)

        return distance

    def closeBy(self, other):
        return self.distance(other)<10

    def inArea(self):
        return self.distance(Position(12.568337, 55.676098))<10000
    
    def getCoordinates(self):
        return (self.x, self.y)

### Our Assets

Here are our bicycles & scooters

In [12]:
#@title Asset

import time
import random


class Asset:
    #lon and lat are the default coordinates for Radhuspladsen
    def __init__(self, lon=12.568337, lat=55.676098, price=0, activation=0, station=1):
        self.position = Position(lon, lat)
        self.locked = True
        self.starting_time = None
        self.needs_repair = False
        self.Station_Id=None
        self.price=price
        self.activation=activation
        self.battery=random.randint(20,100)

    def __str__(self):
        class_name = type(self).__name__
        return (
            f"{class_name} with:\n....(Position: ({str(self.position)}), \n....Locked: {self.locked}, \n....Needs Repair: {self.needs_repair}, \n....Station ID: {self.Station_Id or 'Not assigned'}, \n....Price: ${self.price}, \n....Activation Fee: {self.activation}, \n....Battery: {self.battery}%)\n"
        )



    def unlock(self):
        if self.needs_repair:
            print('unable to unlock: scooter needs repair')
            return False
        elif not self.locked:
            print('unable to unlock: scooter already unlocked')
            return False
        elif self.battery<10:
            print('unable to unlock: scooter needs charging')
            return False
        else:
            if self.Station_Id is not None:
                #remove from station if it was parked
                stations[self.Station_Id].parked.remove(self)
                self.Station_Id=None
            self.locked = False
            # set the starting time to now
            self.starting_time = time.time()
            return True

    def lock(self, pos, endtime=None):
        # check if scooter is in returnable area
        assert pos is not None and isinstance(pos, Position)
        global trips
        self.position=pos
        self.locked = True
        #extra endtime parameter for testing purposes
        if endtime is None:
              duration = time.time() - self.starting_time
        else:
            duration=endtime-self.starting_time
        #battery decreases by 1% every 2 minutes
        self.battery=100-duration/120
        #battery can't go below 0
        if self.battery<0:
            self.battery=0
        trips+=1

        if pos.inArea():
            # check if scooter is in returnable area
            if random.randint(1,100)==1:
              self.needs_repair=True
            # reset for next use
            self.starting_time = None
            for station_id in stations.keys():
                station=stations.get(station_id)
                if pos.closeBy(station.position):
                    station.parked.append(self)
                    self.Station_Id=station_id
                    return (True,True, duration)
            #returns inRange, inStation, duration
            return (True, False, duration)
        else:
            #if scooter is out of bounds, it needs pickup which is implemented as repair
          self.needs_repair=True
          self.starting_time = None
          return (False, False, duration)

class Scooter(Asset):
    #scooter class for different pricing and activation fees
  scooter_activation=2
  scooter_price=0.99
  def __init__(self, lon=12.568337, lat=55.676098):
    super().__init__(lon, lat, self.scooter_price, self.scooter_activation
    )

class Bicycle(Asset):
  bicycle_activation=4
  bicycle_price=0.80
  def __init__(self, lon=12.568337, lat=55.676098):
    super().__init__(lon, lat, self.bicycle_price, self.bicycle_activation)



### Stations

In [13]:
#@title Station

class Station:
    def __init__(self, lon, lat):
        self.parked = [] # asset Ids
        self.position = Position(lon, lat)

    def __str__(self):
        parked_assets = ', '.join(map(str, self.parked)) if self.parked else "None"
        return (
        f"Location(Position: ({str(self.position)}), "
        f"Parked Assets: [{parked_assets}])"
        )


### Sorted, limited, Linked List

To speed up the process of finding close stations, we used a linked list with limited places (25 here) and sorted based on a data attribute.

In [14]:
#@title Node

class Node:
    limit=25
    def __init__(self, asset=-1, distance=-1, number=0):
        self.asset=asset
        self.data = distance
        self.next = None
        self.number=number

    def insert(self, newAsset, newData):
      assert newAsset is not None and newData is not None
      #case if the list is empty
      if self.data==-1:
        self.asset=newAsset
        self.data=newData
        return -1
      
      elif self.data>newData:
        #adds the new data to the current position as it is smaller than the current data there
        temp1=self.asset
        temp2 = self.data
        self.data = newData
        self.asset=newAsset
        newNode = Node(temp1, temp2, self.number)
        newNode.next = self.next
        self.next = newNode
        #shifting all numbers after to account for the new node
        return newNode.shiftNumbering()
      
      #case if the data is larger than the current data and there is a next node
      elif self.next is not None:
        #calls on insert on the next node as the data is larger
        return self.next.insert(newAsset, newData)
      
      #case if the data is larger than the current data and there is no next node
      elif self.number!=self.limit-1:
        #adds it to the end of the list if there aren't 25 elements already
        self.next = Node(newAsset, newData, self.number+1)
        return -1
      
      #case if the data is larger than the current data and there are 25 elements already
      else:
        return self.data


    #shifts the numbering of the nodes after the current node
    def shiftNumbering(self):
      self.number+=1
      
      #if the current node is the last node, it returns the data and sets the next node to None
      if self.number==self.limit-1:
        self.next=None
        return self.data
      
      #if there is a next node, it calls on shiftNumbering on the next node
      if self.next is not None:
        return self.next.shiftNumbering()
      
      #if there is no next node and we did not reach the limit, it returns -1
      return -1


    def __str__(self):
      if self.next is None:
        return f'{self.data}'
      else:
        return f'{self.data}'+'->'+str(self.next)

    def toList(self):
      if self.next is None:
        return [self.asset]
      else:
        return [self.asset]+self.next.toList()


## User Class

Here we also have Payment Method as its own class

In [15]:
#@title User

class PaymentMethod:
    def __init__(self, email, cardholder, cc_number, cvv, expiry_date):
        self.email = email
        self.cardholder = cardholder
        self.cc_number = cc_number
        self.cvv = cvv
        self.expiry_date = expiry_date

    def __str__(self):
        masked_cc_number = f"**** **** **** {self.cc_number[-4:]}"
        masked_cvv = "***"
        return (
        f"PaymentInfo(Email: {self.email}, Cardholder: {self.cardholder}, "
        f"Credit Card: {masked_cc_number}, CVV: {masked_cvv}, Expiry Date: {self.expiry_date})"
        )


class User:
    def __init__(self, phone, lon=12.568337, lat=55.676098):
        self.position = Position(lon, lat)
        self.phone = phone
        self.has_payment_method = None
        self.rented_assets = []
        self.balance = 0
        self.has_membership = False #has_membership benefit is no activation fee

    def update_position(self, lon, lat):
        self.position = Position(lon, lat)

    def add_payment_method(self, email, cardholder, cc_number, cvv, expiry_date):
        self.has_payment_method = PaymentMethod(email, cardholder, cc_number, cvv, expiry_date)

    def add_membership(self):
        if self.add_payment_method:
            self.has_membership = True
            return True
        else:
            return False

    def remove_membership(self):
        if self.has_membership:
            self.has_membership == False
            return True
        else:
           return False

    def find_closest_asset(self):
        #Initializing list, furthest distance
        #we will only give the closest 25 assets
        global assets
        listHead=Node()
        furthest_Distance=-1
        #iterate through assets
        for asset_id in assets.keys():
          asset = assets.get(asset_id)
          if asset.locked and asset.battery>10 and not asset.needs_repair:
            distance=asset.position.distance(self.position)
            if furthest_Distance==-1:
                furthest_Distance=listHead.insert(asset_id, distance)
            elif distance<furthest_Distance:
                furthest_Distance=listHead.insert(asset_id, distance)
        return listHead.toList()

    def find_closest_station(self):
        #Initializing list, furthest distance
        #we will only give the closest 25 stations
        global stations
        listHead=Node()
        furthest_Distance=-1
        #iterate through stations
        for station_id in stations.keys():
          station=stations.get(station_id)
          distance=station.position.distance(self.position)
          if furthest_Distance==-1:
            furthest_Distance=listHead.insert(station_id, distance)
          elif distance<furthest_Distance:
            furthest_Distance=listHead.insert(station_id, distance)
        return listHead.toList()

    def rent_asset(self, asset_id):
        asset = assets.get(asset_id)
        if asset is None:  raise Exception("Asset not found")
        
        # check if conditions are met (payment type added)
        if self.has_payment_method:
            # mark scooter as in use
            isRented = asset.unlock()
            # add scooter to user
            isRented and self.rented_assets.append(asset_id)
            # return status
            return (isRented)
        # else return status
        else:
            return False

    def return_asset(self, asset_id, endtime=None):
        # check for scooters
        asset = assets.get(asset_id)
        if asset is None:  raise Exception("Asset not found")
        
        if asset_id in self.rented_assets:
            # set scooter to default state
            isInRange, inStation, duration = asset.lock(self.position, endtime)
            # remove scooter from user
            self.rented_assets.remove(asset_id)
            
            # calculate extra charge if out of bounds or a discount if in station
            extra_charge = 0
            if not isInRange:
                extra_charge = 100
            if inStation:
                extra_charge=-2
            # calculate price based on has_membership
            if self.has_membership:
                trip_price = duration/60 * asset.price+extra_charge
            else:
                trip_price = duration/60 * asset.price + asset.activation+extra_charge
            trip_price = trip_price if trip_price > 0 else 0
            self.balance += trip_price
        # return status
            return trip_price
        else:
            return False
 
    
    def __str__(self):
        payment_info = str(self.has_payment_method) if self.has_payment_method else "No Payment Method"
        rented = ', '.join(map(str, self.rented_assets)) if self.rented_assets else "None"
        return (
        f"User(Position: ({str(self.position)}), Phone: {self.phone}, "
        f"Payment Method: {payment_info}, "
        f"Rented Scooters: [{rented}], Membership: {'Active' if self.has_membership else 'Inactive'})"
        )


## Testing

First we create the test environment.

In [16]:
import random
random.seed(77)

def random_pos(lat_offset = 0.0898, lon_offset = 0.1568):
    # Define the center point
    center_lon = 12.568337
    center_lat = 55.676098

    # Generate random longitude and latitude within the defined range
    random_lon = random.uniform(center_lon - lon_offset, center_lon + lon_offset)
    random_lat = random.uniform(center_lat - lat_offset, center_lat + lat_offset)

    return (random_lon, random_lat)


def random_time_addition():

    # Using an exponential distribution with a mean of 30 minutes (1800 seconds)
    seconds_to_add = min(int(random.expovariate(1/1800)), 43200)  # Cap at 12 hours (43200 seconds)
    
    return time.time() + seconds_to_add


# Create Stations
def create_random_stations(count):
    global stations
    stations.update({"1": Station(12.568337,55.676098)})
    for i in range(2, count):
        lon, lat = random_pos()
        stations.update({str(i): Station(lon, lat)})

create_random_stations(100)
print(stations.get('1'))

# Create Scooters
# In 'assets' Bycicles and Scooters are implemented in the same manner and behave the same.
def create_random_scooters(count):
    for i in range(1, count):
        lon, lat = random_pos()
        assets.update({"S"+str(i): Scooter(lon, lat)})

create_random_scooters(5000)

def create_random_bicycles(count):
    for i in range(1, count):
        lon, lat = random_pos()
        assets.update({"B"+str(i): Bicycle(lon, lat)})

create_random_bicycles(5000)
print(assets.get('B1'))
print(assets.get('S2'))
print(assets.get('B3'))
print(assets.get('S4'))
# Create User
def create_random_user(count):
    for i in range(1, count):
        lon, lat = random_pos()
        user = User(f'+45{random.randint(100000,999999)}', lon, lat)
        temp = {str(i): user}
        users.update(temp)

create_random_user(10000)
print(users.get('1'))

Location(Position: (X: 12.568337, Y: 55.676098), Parked Assets: [None])
Bicycle with:
....(Position: (X: 12.526992104207856, Y: 55.640775803363645), 
....Locked: True, 
....Needs Repair: False, 
....Station ID: Not assigned, 
....Price: $0.8, 
....Activation Fee: 4, 
....Battery: 38%)

Scooter with:
....(Position: (X: 12.581776905485425, Y: 55.706352719678826), 
....Locked: True, 
....Needs Repair: False, 
....Station ID: Not assigned, 
....Price: $0.99, 
....Activation Fee: 2, 
....Battery: 28%)

Bicycle with:
....(Position: (X: 12.494067181367521, Y: 55.7031134676328), 
....Locked: True, 
....Needs Repair: False, 
....Station ID: Not assigned, 
....Price: $0.8, 
....Activation Fee: 4, 
....Battery: 40%)

Scooter with:
....(Position: (X: 12.571979949534581, Y: 55.599147457684396), 
....Locked: True, 
....Needs Repair: False, 
....Station ID: Not assigned, 
....Price: $0.99, 
....Activation Fee: 2, 
....Battery: 81%)

User(Position: (X: 12.715226511055837, Y: 55.69421830813321), Phone:

Next, we go through two possible customer journeys.

In [ ]:


# User journey
max = users.get('245')
moritz = users.get('345')
print(max)
print(moritz)

# Add Payment Method
max.add_payment_method('max@moritz.org','max mustermann', '000000000', '231', '02/28')
moritz.add_payment_method('moritz@max.org','moritz samplesen', '100000000', '331', '04/28')

# Find nearest Scooter
closest_assets = max.find_closest_asset()

# test if distances are close
print('Asset: checking location distances:')
for asset in closest_assets:
    print(f'....{asset}: {assets.get(asset).position.distance(max.position)}')

# Rent Scooter
# max takes the closest bike
max.rent_asset(closest_assets[0])
# max zooms to his lecture at sp 
max.update_position(12.527218, 55.681576)


# Find Nearest Station
# max looks for the closest station
closest_stations = max.find_closest_station()
closest_station = closest_stations[0]
print('Station: checking location distances:')
for station in closest_stations:
    print(f'....{station}: {stations.get(station).position.distance(max.position)}')

# Set user location to nearest station
print('max rides to the closest station')
x,y = stations.get(closest_station).position.getCoordinates()
max.update_position(x,y)

# Return Scooter
print('max returns his scooter')
rented_asset = max.rented_assets[0]
return_time = random_time_addition()
max_paid = round(max.return_asset(rented_asset, return_time),2)
print(f'Overall, Max rented for {round((return_time-time.time())/60,1)} minutes')
print(f'ching ching we made {max_paid} dkk')

# get membership
moritz.add_membership()
print('moritz gets a membership')
moritz_closest_assets = moritz.find_closest_asset()
# Rent Scooter
moritz.rent_asset(moritz_closest_assets[0])
print('moritz takes the closest scooter')
moritz.update_position(random_pos()[0], random_pos()[1])
# Return Scooter not at station
timeadded= random_time_addition()
print('moritz returns his scooter')
moritz_paid = round(moritz.return_asset(moritz.rented_assets[0], timeadded),2)
print(f'Moritz took a scooter and returned it after {round((timeadded-time.time())/60,1)} minutes')
print(f'due to his membership benefits the price for moritz is: {moritz_paid} dkk')

User(Position: (X: 12.503330914637651, Y: 55.7069142707324), Phone: +45263852, Payment Method: No Payment Method, Rented Scooters: [None], Membership: Inactive)
User(Position: (X: 12.671346328058677, Y: 55.613116005175385), Phone: +45150387, Payment Method: No Payment Method, Rented Scooters: [None], Membership: Inactive)
Asset: checking location distances:
....B3966: 67.97039161215827
....B883: 87.48077626187164
....B4220: 118.93272961738349
....B834: 173.69855799426654
....B4790: 180.76633921260856
....B643: 214.05046341914166
....S1397: 273.78035245733975
....B1055: 310.1395285701946
....B782: 337.62861955307636
....B3735: 339.3931906892802
....B2536: 349.5333177323237
....S1838: 366.1733817608507
....S870: 431.34509956137373
....S2395: 459.5266197465999
....B417: 463.0753917662186
....B1571: 487.03081190746354
....B2537: 487.93312942192904
....B948: 519.9263525102831
....S2737: 530.1045897351046
....B3785: 554.9021003827183
....B1749: 555.1397146557789
....B3926: 560.0033762274979


True

To end, we have a larger scale simulation:

In [18]:

for i in range(1, 10):
    for k in range(1, 10):
        for j in range(1, 10):
            user= users.get(str(random.randint(1,9999)))
            if user.has_payment_method is None:
                user.add_payment_method(f'{random.randint(100000,999999)}@{random.randint(100000,999999)}.com', f'{random.randint(100000,999999)} {random.randint(100000,999999)}', f'{random.randint(1000000000000000,9999999999999999)}', f'{random.randint(100,999)}', f'{random.randint(1,12)}/{random.randint(24,30)}')
            if random.randint(1,100)==1:
                if user.has_membership:
                    user.remove_membership()
                else:
                    user.add_membership()
                    
            user.update_position(random_pos()[0], random_pos()[1])
            
            no= random.randint(0,4)
            for l in range(0, no):
                user.rent_asset(user.find_closest_asset()[l])
                
            if random.randint(1,5)==1:
                #20% chance to return the scooter within a 20km radius, so potentially outside of zone
                user.update_position(random_pos(0.1796, 0.3136)[0], random_pos(0.1796, 0.3136)[1])
            else:
                user.update_position(random_pos()[0], random_pos()[1])
            
            if random.randint(1,5)==1:
                x,y = stations.get(user.find_closest_station()[0]).position.getCoordinates()
                max.update_position(x,y)
                
            for l in range(0, no):
                user.return_asset(user.rented_assets[0], random_time_addition())
                
        chargeAll()
          
    repairAll()

sum=0
for user in users.values():
    sum+=user.balance
    
print(f'With our {trips} trips {round(sum,2)} dkk were made, with {repairs} repairs and {charges} battery charges')
 

With our 1479 trips 97203.44 dkk were made, with 537 repairs and 81 battery charges
